# Completing actor data

In [11]:
from src.data.transform_data import* 
from src.models.movie_success_model import* 
from src.models.actor_success_model import*
from webscraping.webscraper import*

## 1) Load actor data from csv

In [12]:
actor_df = pd.read_csv('data/actor_data.csv', index_col=0)
actor_df

,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index
Actor name,,,,,,
denzel_washington,1954-12-28,M,1.84,/m/0x67,26.0,10.000000
matt_damon,1970-10-08,M,1.78,/m/0cnvdq1,18.0,9.912690
tom_hanks,1956-07-09,M,1.83,/m/02p4q5p,27.0,9.741555
eddie_murphy,1961-04-03,M,1.75,/m/0x67,21.0,9.415060
tom_cruise,1962-07-03,M,1.70,/m/02ctzb,18.0,9.362255
...,...,...,...,...,...,...
j._kenneth_campbell,1947-07-22,M,NaN,NaN,43.0,0.714754
evelyn_keyes,1916-11-20,F,1.63,NaN,72.0,0.700949
james_dixon,NaN,M,NaN,NaN,NaN,0.700949


## 2) Set sample size

In [13]:
test_sample = actor_df[:20]
test_sample

,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index
Actor name,,,,,,
denzel_washington,1954-12-28,M,1.840,/m/0x67,26.0,10.000000
matt_damon,1970-10-08,M,1.780,/m/0cnvdq1,18.0,9.912690
tom_hanks,1956-07-09,M,1.830,/m/02p4q5p,27.0,9.741555
eddie_murphy,1961-04-03,M,1.750,/m/0x67,21.0,9.415060
tom_cruise,1962-07-03,M,1.700,/m/02ctzb,18.0,9.362255
frank_welker,1946-03-12,M,1.730,NaN,38.0,8.787846
harrison_ford,1942-07-13,M,1.850,/m/01qhm_,38.0,8.450475
james_woods,1947-04-18,M,1.803,NaN,35.0,8.180804
michelle_pfeiffer,1958-04-29,F,1.710,/m/065b6q,21.0,8.174009


## 3) Run the scraper

In [14]:
spider = ActorScraper(test_sample)
test_sample = spider.run_scraping()
test_sample

/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:314: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'Gender'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'University'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:316: SettingWithCopyWarning: 
A value 

100%|██████████| 20/20 [00:51<00:00,  2.59s/it]

Scraping completed.


,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index,Gender,University,Theater,Sports,Birth City,Date of Birth,Citizenship,Number of Children,Career Start
Actor name,,,,,,,,,,,,,,,
Denzel Washington,1954-12-28,M,1.840,/m/0x67,26.0,10.000000,Male,Fordham University BAAmerican Conservatory The...,Yes,Basketball,Mount Vernon New York US,1954-12-28,None,4,1975
Matt Damon,1970-10-08,M,1.780,/m/0cnvdq1,18.0,9.912690,Male,Harvard University dropped out,Yes,None,Cambridge Massachusetts US,1970-10-08,None,4,1987
Tom Hanks,1956-07-09,M,1.830,/m/02p4q5p,27.0,9.741555,Male,Chabot College,Yes,None,Concord California US,1956-07-09,"United States, Greece1",4,1977
Eddie Murphy,1961-04-03,M,1.750,/m/0x67,21.0,9.415060,Male,None,None,None,New York City US,1961-04-03,None,10,1976
Tom Cruise,1962-07-03,M,1.700,/m/02ctzb,18.0,9.362255,Male,None,None,Football,Syracuse New York US,1962-07-03,None,3,1980
Frank Welker,1946-03-12,M,1.730,NaN,38.0,8.787846,Male,Santa Monica College,None,None,Denver Colorado US,1946-03-12,None,None,1967
Harrison Ford,1942-07-13,M,1.850,/m/01qhm_,38.0,8.450475,Male,None,None,None,Chicago Illinois US,1942-07-13,None,5,1964
James Woods,1947-04-18,M,1.803,NaN,35.0,8.180804,None,None,Yes,None,Vernal Utah US,1947-04-18,None,None,1969
Michelle Pfeiffer,1958-04-29,F,1.710,/m/065b6q,21.0,8.174009,Female,None,None,None,Santa Ana California US,1958-04-29,None,2,1978
